## Description
This notebook will introduce how to load data from the datacube through the construction of a query and use of the dc.load_ard() function. Topics covered include:

* Loading data using dc.load_ard()
* Interpreting the resulting xarray.Dataset object
* Inspecting an individual xarray.DataArray
* Customising parameters passed to the dc.load_ard() function
* Loading specific measurements
* Loading data for coordinates in a custom coordinate reference system (CRS)
* Loading data using a reusable dictionary query
* Adding a progress bar to the data load

In [1]:
import datacube
import sys
sys.path.insert(1, '../tools/')
from odc.ui import with_ui_cbk
from tools.datahandling import load_ard, mostcommon_crs

In [2]:
dc = datacube.Datacube(app='Sentinel_2')

In [3]:
lat, lon = 30.5, 31.7
buffer = 0.125

# Create a reusable query
query = {
    'x': (lon-buffer, lon+buffer),
    'y': (lat+buffer, lat-buffer),
    'time': ('2022-01-01', '2022-01-31'),
    'resolution': (-20, 20)
}

# Identify the most common projection system in the input query
output_crs = mostcommon_crs(dc=dc, product='s2_l2a', query=query)
print(output_crs)

# Load available data from Sentinel-2 and filter to retain only times
# with at least 90% good data
ds = load_ard(dc=dc, 
              products=['s2_l2a'],
              min_gooddata=0.9, 
              measurements=['red', 'green', 'blue', 'swir_1', 'swir_2', 'nir', 'nir_2'],
              output_crs=output_crs,
              progress_cbk=with_ui_cbk(),
              **query)

epsg:32636


Using pixel quality parameters for Sentinel 2
Finding datasets
    s2_l2a
Counting good quality pixels for each time step
Filtering to 4 out of 6 time steps with at least 90.0% good quality pixels
Applying pixel quality/cloud mask
Loading 4 time steps


In [4]:
print(ds)

<xarray.Dataset>
Dimensions:      (time: 4, y: 1400, x: 1217)
Coordinates:
  * time         (time) datetime64[ns] 2022-01-03T08:41:48 ... 2022-01-23T08:...
  * y            (y) float64 3.389e+06 3.389e+06 ... 3.361e+06 3.361e+06
  * x            (x) float64 3.631e+05 3.631e+05 ... 3.874e+05 3.874e+05
    spatial_ref  int32 32636
Data variables:
    red          (time, y, x) float32 286.0 343.0 401.0 ... 2.43e+03 2.139e+03
    green        (time, y, x) float32 823.0 788.0 777.0 ... 1.921e+03 1.636e+03
    blue         (time, y, x) float32 403.0 419.0 425.0 ... 1.254e+03 1.063e+03
    swir_1       (time, y, x) float32 2.087e+03 1.839e+03 ... 3.189e+03
    swir_2       (time, y, x) float32 1.027e+03 910.0 ... 3.536e+03 3.133e+03
    nir          (time, y, x) float32 5.526e+03 4.561e+03 ... 2.786e+03
    nir_2        (time, y, x) float32 5.517e+03 4.555e+03 ... 2.773e+03
Attributes:
    crs:           epsg:32636
    grid_mapping:  spatial_ref


In [5]:
print(ds.red)

<xarray.DataArray 'red' (time: 4, y: 1400, x: 1217)>
array([[[ 286.,  343.,  401., ..., 1052.,  725.,  698.],
        [ 287.,  369.,  355., ...,  594.,  727.,  697.],
        [ 423.,  315.,  351., ...,  629.,  674.,  606.],
        ...,
        [1848., 2380., 1516., ..., 2461., 2177.,  831.],
        [2168., 1773., 1964., ..., 2029., 2296., 1318.],
        [2361., 2038., 1904., ..., 2665., 2093., 1803.]],

       [[ 174.,  251.,  289., ..., 1063.,  653.,  594.],
        [ 197.,  326.,  202., ...,  485.,  657.,  622.],
        [ 364.,  221.,  200., ...,  539.,  651.,  519.],
        ...,
        [1661., 2114., 1383., ..., 2528., 2135.,  708.],
        [1973., 1630., 1888., ..., 2097., 2351., 1314.],
        [2249., 1989., 1906., ..., 2655., 2150., 1795.]],

       [[ 208.,  237.,  286., ..., 1231.,  705.,  572.],
        [ 211.,  275.,  194., ...,  431.,  582.,  666.],
        [ 344.,  255.,  172., ...,  514.,  605.,  526.],
        ...,
        [1733., 2490., 1396., ..., 2501., 2193., 